# krx 종목 코드 xlsx file

In [4]:
#krx에 상장된 종목 코드파일 불러오기
stock_code = pd.read_excel('krx_stockcode.xlsx')

In [5]:
#상장일이 2009년6월 이후인 종목만 고르기
stock_code = stock_code[stock_code['상장일']> '2009-06'].copy()
stock_code.set_index('상장일')
temp = pd.DataFrame(stock_code.reset_index())
stock_code = temp.drop('index',axis=1).copy()
stock_code['종목코드'].astype(np.str)
stock_code['상장일']=pd.to_datetime(stock_code['상장일'])
#rcp_no 컬럼 생성
stock_code['rcp_no']=''

# stock_code -> rcp_no crawling -> xlsx file 저장

In [3]:
#개별 종목의 상장하기 가장 인접한 시기에 공시한 투자설명서의 rcp_no 가져오기
import requests
import json
#from konlpy.utils import pprint
import re
import pandas as pd
import numpy as np
#코드함수화
def getting_rcp_no(stock_code):
    for i, code in enumerate(stock_code['종목코드']):
        a = str(stock_code.loc[i,'상장일'].date())
        temp = a.split('-')
        date = temp[0]+temp[1]+temp[2]
        url =  'http://dart.fss.or.kr/api/search.json?auth=ac65dd5e279576ba95b9459d8f67cf4dfca016a2'+'&crp_cd='+ str(code)+'&start_dt='+'19990101'+'&end_dt='+ str(date)+'&bsn_tp=C001&bsn_tp=C002&bsn_tp=C003&bsn_tp=C004&bsn_tp=C005'
        response = requests.get(url)
        data = json.loads(response.content)
        k = re.compile('[정정] 투자설명서|투자설명서$')
        for s in range(len(data['list'])):
            if k.findall( data['list'][s]['rpt_nm']):
                stock_code.loc[i,'rcp_no']= data['list'][s]['rcp_no']
                break
            elif s == len(data['list'])-1:
                stock_code.loc[i,'rcp_no'] = 'non'
                print(str(code)+'얘는 투자설명서 없다')
                break
            else:
                pass
               
                
    return stock_code

In [6]:
#데이터 프레임으로 받아오기
df= getting_rcp_no(stock_code)
#rcp_no 가 공백이 아닌 행만 가져와서 데이터프레임으로 저장
df_master=df[df['rcp_no']!='non']
df_master = df_master[df['rcp_no']!='']
len(df_master)

101730얘는 투자설명서 없다
102120얘는 투자설명서 없다


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


521

In [ ]:
#엑셀파일로 던져주기
dir= 'rcp_no.xlsx'
df_master.to_excel(dir)

# rcp_no.xlsx import -> section parameter crawling -> section_parameter_df.xlsx file 저장 

In [7]:
import pandas as pd
import numpy as np
import requests
import re
import bs4
from urllib.request import urlopen
from time import sleep

import rcp_no csv file

In [8]:
df_rcp_no = pd.read_excel('rcp_no.xlsx')
print(len(df_rcp_no))
df_rcp_no.head()

521


,회사명,종목코드,업종,상장일,결산월,rcp_no
6,삼성바이오로직스,207940,기초 의약물질 및 생물학적 제제 제조업,2016-11-10,12월,20161028000334
9,우진,105840,"측정, 시험, 항해, 제어 및 기타 정밀기기 제조업; 광학기기 제외",2010-07-26,12월,20100709000281
10,웅진에너지,103130,반도체 제조업,2010-06-30,12월,20100618000008
14,제이에스코퍼레이션,194370,"가죽, 가방 및 유사제품 제조업",2016-02-04,12월,20160125000066
20,코오롱플라스틱,138490,합성고무 및 플라스틱 물질 제조업,2011-06-15,12월,20110531000084


# 모든 종목의 문서에 대한 투자위험요소 section parameter 추출

In [9]:
def getting_parameter_url(df_rcp_no):
    sleep(2)
    url = 'http://dart.fss.or.kr/dsaf001/main.do?'
    rcp_no_list = list(df_rcp_no['rcp_no'])
    rcp=[]
    dcm = []
    eleId=[] 
    offset =[] 
    length = []
    dtd = []
    df = pd.DataFrame()
    for rcp_no in rcp_no_list: #rcp_no list 로 받아옴
        real_url = url+'rcpNo='+str(rcp_no) #문자형으로 변환
        r = urlopen(real_url) #투자설명서 요청 url 
        soup = bs4.BeautifulSoup(r,'html.parser')
        print( str(rcp_no)+'html')
        script_list = soup.find_all('script') #모든 스크립트 태그
        print(str(rcp_no)+'script')
        target_script = script_list[7] #타겟 스크립트 태그
        print(str(rcp_no)+'타켓 스크립트 태그')
        script_text = target_script.text #타겟 스크립트 태그의 텍스트
        print(str(rcp_no)+'타겟 스크립트 태그의 텍스트')
        target_text = script_text.replace("\t", " ").split("//")[10] #본문 부분 텍스트
        print(str(rcp_no)+'본문 부분 텍스트')
        if 'viewDoc' in target_text:
            final_text = target_text.split('});')[7] #투자위험요소 부분
            print(str(rcp_no)+'투자위험요소부분 추출')
        else:
            target_text = script_text.replace("\t", " ").split("//")[9]
            final_text = script_text.split('});')[7]
            print(str(rcp_no)+'투자위험요소부분 추출')
            
        pattern =re.compile('viewDoc(.+)') #parameter 부분 
        if pd.isna(pattern.findall(final_text)[0]):
            pass
        else:
            a = pattern.findall(final_text)[0] #리스트 풀어주기
            print(a)
            parameter = re.sub('[^a-zA-Z0-9,]','', a).split(',') #parameter 추출final_text = target_text.split('});')[7] #투자위험요소 부분
        
            rcp.append(parameter[0])
            dcm.append(parameter[1])
            eleld.append(parameter[2])
            offset.append(parameter[3])
            length.append(parameter[4])
            dtd.append(parameter[5])
        
            df['rcp_no']=rcp
            df['dcm'] = dcm
            df['eleld']= eleld
            df['offset']= offset
            df['length'] = length
            df['dtd'] = dtd
    return df

function test

In [81]:
col = ['rcp_no','dcm','eleld','offset','length','dtd']
df = pd.DataFrame(columns = col)
def getting_parameter_temp(df_rcp_no):
    sleep(2)
    url = 'http://dart.fss.or.kr/dsaf001/main.do?'
    rcp_no_list = list(df_rcp_no['rcp_no'])
    for i,rcp_no in enumerate(rcp_no_list): #rcp_no list 로 받아옴
        real_url = url+'rcpNo='+str(rcp_no) #문자형으로 변환
        r = urlopen(real_url) #투자설명서 요청 url 
        soup = bs4.BeautifulSoup(r,'html.parser')
        print( str(rcp_no)+'html')
        script_list = soup.find_all('script') #모든 스크립트 태그
        print(str(rcp_no)+'script')
        target_script = script_list[7] #타겟 스크립트 태그
        print(str(rcp_no)+'타켓 스크립트 태그')
        script_text = target_script.text #타겟 스크립트 태그의 텍스트
        print(str(rcp_no)+'타겟 스크립트 태그의 텍스트')
        
        if 'Ⅲ. 투자위험요소' in script_text.replace("\t", " ").split("//")[10]:
            target_text = script_text.replace("\t", " ").split("//")[10] #본문 부분 텍스트
            print(str(rcp_no)+'본문 부분 텍스트')
            final_text = target_text.split('});')[7] #투자위험요소 부분
            print(str(rcp_no)+'투자위험요소부분 추출')
            
        elif 'Ⅲ. 투자위험요소' in script_text.replace("\t", " ").split("//")[9]:
            target_text = script_text.replace("\t", " ").split("//")[9]
            print(str(rcp_no)+'본문 부분 텍스트')
            final_text = target_text.split('});')[7]
            print(str(rcp_no)+'투자위험요소부분 추출')
            
        elif 'Ⅲ. 투자위험요소' in  script_text.replace("\t", " ").split("//")[8]:
            target_text= script_text.replace("\t", " ").split("//")[8]
            print(str(rcp_no)+'본문 부분 텍스트')
            final_text = target_text.split('});')[7]
            print(str(rcp_no)+'투자위험요소부분 추출')
            
        elif 'Ⅲ. 투자위험요소' not in script_text.replace("\t", " ").split("//")[9]:
            print(str(rcp_no)+'이 투자설명서에는 정보가 없다')
            pass
                
        if 'Ⅲ. 투자위험요소' not in script_text.replace("\t", " ").split("//")[9]:
            df.loc[i,'rcp_no']= rcp_no
            df.loc[i,'dcm'] = 'non'
            df.loc[i,'eleld']= 'non'
            df.loc[i,'offset']= 'non'
            df.loc[i,'length'] = 'non'
            df.loc[i,'dtd'] = 'non'
        else:
            a = pattern.findall(final_text)[0] #리스트 풀어주기
            parameter = re.sub('[^a-zA-Z0-9,]','', a).split(',') #parameter 추출final_text = target_text.split('});')[7] #투자위험요소 부분
            df.loc[i,'rcp_no']=parameter[0]
            df.loc[i,'dcm'] = parameter[1]
            df.loc[i,'eleld']= parameter[2]
            df.loc[i,'offset']= parameter[3]
            df.loc[i,'length'] = parameter[4]
            df.loc[i,'dtd'] = parameter[5]
    print('parameter crawling 끝!')

debugging try

In [37]:
'''url = 'http://dart.fss.or.kr/dsaf001/main.do?'
rcp_no_list = list(df_rcp_no['rcp_no'])
rcp=[]
dcm = []
eleld=[]
offset =[] 
length = []
dtd = []
df = pd.DataFrame()
real_url = url+'rcpNo='+str(20100709000281)
r = urlopen(real_url) #투자설명서 요청 url 
soup = bs4.BeautifulSoup(r,'html.parser')
script_list = soup.find_all('script') #모든 스크립트 태그
target_script = script_list[7] #타겟 스크립트 태그
script_text = target_script.text #타겟 스크립트 태그의 텍스트'''

In [38]:
'''target_text = script_text.replace("\t", " ").split("//")[10] #본문 부분 텍스트

if 'viewDoc' in target_text:
    final_text = target_text.split('});')[7] #투자위험요소 부분
else:
    target_text = script_text.replace("\t", " ").split("//")[9]
    final_text = target_text.split('});')[7] #투자위험요소 부분
    
pattern =re.compile('viewDoc(.+)') #parameter 부분 
a = pattern.findall(final_text)[0] #리스트 풀어주기
parameter = re.sub('[^a-zA-Z0-9,]','', a).split(',') #parameter 추출final_text = target_text.split('});')[7] #투자위험요소 부분
rcp.append(parameter[0])
dcm.append(parameter[1])
eleld.append(parameter[2])
offset.append(parameter[3])
length.append(parameter[4])
dtd.append(parameter[5])
df['rcp_no']=rcp
df['dcm'] = dcm
df['eleld']= eleld
df['offset']= offset
df['length'] = length
df['dtd'] = dtd
df'''

,rcp_no,dcm,eleld,offset,length,dtd
0,20100709000281,2723113,16,302030,46836,dart3xsd


In [28]:
'''url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20100621000044'
r = urlopen(url) #투자설명서 요청 url 
soup = bs4.BeautifulSoup(r,'html.parser')
script_list = soup.find_all('script') #모든 스크립트 태그
target_script = script_list[7] #타겟 스크립트 태그
script_text = target_script.text #타겟 스크립트 태그의 텍스트
target_text= script_text.replace("\t", " ").split("//")[8]
final_text = target_text.split('});')[7]
final_text'''

'\r\n  treeNode2.appendChild(treeNode3);\r\n  cnt++;\r\n   \r\n  treeNode3 = new Tree.TreeNode({\r\n   text: "Ⅲ. 투자위험요소",\r\n   id: "15",\r\n   cls: "text",\r\n   listeners: {\r\n    click: function() {viewDoc(\'20100621000044\', \'2707897\', \'15\', \'147342\', \'58993\', \'dart3.xsd\');}\r\n   }\r\n  '

In [19]:
'''target_text = script_text.replace("\t", " ").split("//")[10] #본문 부분 텍스트
if 'viewDoc' in target_text:
    final_text = target_text.split('});')[7] #투자위험요소 부분
else:
    target_text = script_text.replace("\t", " ").split("//")[9]
    final_text = target_text.split('});')[7]
pattern =re.compile('viewDoc(.+)') #parameter 부분 
a = pattern.findall(final_text)[0] #리스트 풀어주기
re.sub('[^a-zA-Z0-9,]','', a).split(',') #parameter 추출final_text = target_text.split('});')[7] #투자위험요소 부분'''

'\r\n\r\nvar winTestNotice = null;\t// xwindow object\r\nvar divTestNotice = "divTestNoticeWin";\t// ajax 수신 div\r\n$j(document).ready(function() {\r\n\tinitPage();\r\n\r\n\tinitScroll();\r\n\t$j(window).resize(function(){\r\n\t\t$j("#center-panel > div > div").resize(resizeFrame);\r\n\t\tresizeFrame();\r\n\t}).resize();\r\n});\r\n\r\n\r\nvar resizeFrame = function() {\r\n\tw\t= parseInt($j("#center-panel > div > div").css("width"));\r\n\th\t= parseInt($j("#center-panel > div > div").css("height"));\r\n\t//$j("#ifrm").css("width", (w - 12) + "px");\r\n\t\r\n\t\t\r\n\t\r\n\t$j("#ifrm").css("height", h + "px");\r\n\t$j("#mobileScroll").css("height", h + "px");\r\n};\r\n\r\nvar west;\r\n\r\n//모바일(ios)에서 본문내용 스크롤 추가\r\nfunction initScroll() {\r\n\tvar iosKeywords = new Array("iPhone","iPod","iPad");\r\n\tfor(var i=0; i<iosKeywords.length; i++){\r\n\t\tif(navigator.userAgent.indexOf(iosKeywords[i]) > -1) {\r\n\t\t\t$j("#mobileScroll").css("overflow","auto");\r\n\t\t\t$j("#mobileScroll").css

In [53]:
'''url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20090722000032'
r = urlopen(url) #투자설명서 요청 url 
soup = bs4.BeautifulSoup(r,'html.parser')
script_list = soup.find_all('script') #모든 스크립트 태그
target_script = script_list[7] #타겟 스크립트 태그
script_text = target_script.text #타겟 스크립트 태그의 텍스트
target_text= script_text.replace("\t", " ").split("//")[8]
target_text'''

' 3\r\n  treeNode1 = new Tree.TreeNode({\r\n   text: "【 대표이사 등의 확인 】",\r\n   id: "3",\r\n   cls: "text",\r\n   listeners: {\r\n    click: function() {viewDoc(\'20090722000032\', \'2363550\', \'3\', \'132894\', \'356\', \'dart3.xsd\');}\r\n   }\r\n  });\r\n  cnt++;\r\n  \r\n  treeRoot.appendChild(treeNode1);\r\n  \r\n \r\n  '

# parameter 추출

In [82]:
getting_parameter_temp(df_rcp_no)

20161028000334html
20161028000334script
20161028000334타켓 스크립트 태그
20161028000334타겟 스크립트 태그의 텍스트
20161028000334본문 부분 텍스트
20161028000334투자위험요소부분 추출
20100709000281html
20100709000281script
20100709000281타켓 스크립트 태그
20100709000281타겟 스크립트 태그의 텍스트
20100709000281본문 부분 텍스트
20100709000281투자위험요소부분 추출
20100618000008html
20100618000008script
20100618000008타켓 스크립트 태그
20100618000008타겟 스크립트 태그의 텍스트
20100618000008본문 부분 텍스트
20100618000008투자위험요소부분 추출
20160125000066html
20160125000066script
20160125000066타켓 스크립트 태그
20160125000066타겟 스크립트 태그의 텍스트
20160125000066본문 부분 텍스트
20160125000066투자위험요소부분 추출
20110531000084html
20110531000084script
20110531000084타켓 스크립트 태그
20110531000084타겟 스크립트 태그의 텍스트
20110531000084본문 부분 텍스트
20110531000084투자위험요소부분 추출
20170118000323html
20170118000323script
20170118000323타켓 스크립트 태그
20170118000323타겟 스크립트 태그의 텍스트
20170118000323본문 부분 텍스트
20170118000323투자위험요소부분 추출
20130722000030html
20130722000030script
20130722000030타켓 스크립트 태그
20130722000030타겟 스크립트 태그의 텍스트
20130722000030본문 부분 텍스트
20130722000

20150616000146html
20150616000146script
20150616000146타켓 스크립트 태그
20150616000146타겟 스크립트 태그의 텍스트
20150616000146본문 부분 텍스트
20150616000146투자위험요소부분 추출
20151030000357html
20151030000357script
20151030000357타켓 스크립트 태그
20151030000357타겟 스크립트 태그의 텍스트
20151030000357본문 부분 텍스트
20151030000357투자위험요소부분 추출
20170424000109html
20170424000109script
20170424000109타켓 스크립트 태그
20170424000109타겟 스크립트 태그의 텍스트
20170424000109본문 부분 텍스트
20170424000109투자위험요소부분 추출
20100719000083html
20100719000083script
20100719000083타켓 스크립트 태그
20100719000083타겟 스크립트 태그의 텍스트
20100719000083본문 부분 텍스트
20100719000083투자위험요소부분 추출
20171128000038html
20171128000038script
20171128000038타켓 스크립트 태그
20171128000038타겟 스크립트 태그의 텍스트
20171128000038본문 부분 텍스트
20171128000038투자위험요소부분 추출
20140728000039html
20140728000039script
20140728000039타켓 스크립트 태그
20140728000039타겟 스크립트 태그의 텍스트
20140728000039본문 부분 텍스트
20140728000039투자위험요소부분 추출
20150629000424html
20150629000424script
20150629000424타켓 스크립트 태그
20150629000424타겟 스크립트 태그의 텍스트
20150629000424본문 부분 텍스트
20150629000

20170112000089html
20170112000089script
20170112000089타켓 스크립트 태그
20170112000089타겟 스크립트 태그의 텍스트
20170112000089본문 부분 텍스트
20170112000089투자위험요소부분 추출
20170327000498html
20170327000498script
20170327000498타켓 스크립트 태그
20170327000498타겟 스크립트 태그의 텍스트
20170327000498본문 부분 텍스트
20170327000498투자위험요소부분 추출
20150331001027html
20150331001027script
20150331001027타켓 스크립트 태그
20150331001027타겟 스크립트 태그의 텍스트
20150331001027본문 부분 텍스트
20150331001027투자위험요소부분 추출
20181212000279html
20181212000279script
20181212000279타켓 스크립트 태그
20181212000279타겟 스크립트 태그의 텍스트
20181212000279본문 부분 텍스트
20181212000279투자위험요소부분 추출
20161018000162html
20161018000162script
20161018000162타켓 스크립트 태그
20161018000162타겟 스크립트 태그의 텍스트
20161018000162본문 부분 텍스트
20161018000162투자위험요소부분 추출
20140121000129html
20140121000129script
20140121000129타켓 스크립트 태그
20140121000129타겟 스크립트 태그의 텍스트
20140121000129본문 부분 텍스트
20140121000129투자위험요소부분 추출
20100126000064html
20100126000064script
20100126000064타켓 스크립트 태그
20100126000064타겟 스크립트 태그의 텍스트
20100126000064본문 부분 텍스트
20100126000

20111213000156html
20111213000156script
20111213000156타켓 스크립트 태그
20111213000156타겟 스크립트 태그의 텍스트
20111213000156본문 부분 텍스트
20111213000156투자위험요소부분 추출
20150810000107html
20150810000107script
20150810000107타켓 스크립트 태그
20150810000107타겟 스크립트 태그의 텍스트
20150810000107본문 부분 텍스트
20150810000107투자위험요소부분 추출
20181119000018html
20181119000018script
20181119000018타켓 스크립트 태그
20181119000018타겟 스크립트 태그의 텍스트
20181119000018본문 부분 텍스트
20181119000018투자위험요소부분 추출
20170914000418html
20170914000418script
20170914000418타켓 스크립트 태그
20170914000418타겟 스크립트 태그의 텍스트
20170914000418본문 부분 텍스트
20170914000418투자위험요소부분 추출
20181130000313html
20181130000313script
20181130000313타켓 스크립트 태그
20181130000313타겟 스크립트 태그의 텍스트
20181130000313본문 부분 텍스트
20181130000313투자위험요소부분 추출
20100817000164html
20100817000164script
20100817000164타켓 스크립트 태그
20100817000164타겟 스크립트 태그의 텍스트
20100817000164본문 부분 텍스트
20100817000164투자위험요소부분 추출
20131112000010html
20131112000010script
20131112000010타켓 스크립트 태그
20131112000010타겟 스크립트 태그의 텍스트
20131112000010본문 부분 텍스트
20131112000

20150930000232html
20150930000232script
20150930000232타켓 스크립트 태그
20150930000232타겟 스크립트 태그의 텍스트
20150930000232본문 부분 텍스트
20150930000232투자위험요소부분 추출
20151124000081html
20151124000081script
20151124000081타켓 스크립트 태그
20151124000081타겟 스크립트 태그의 텍스트
20151124000081본문 부분 텍스트
20151124000081투자위험요소부분 추출
20170419000074html
20170419000074script
20170419000074타켓 스크립트 태그
20170419000074타겟 스크립트 태그의 텍스트
20170419000074본문 부분 텍스트
20170419000074투자위험요소부분 추출
20181015000183html
20181015000183script
20181015000183타켓 스크립트 태그
20181015000183타겟 스크립트 태그의 텍스트
20181015000183본문 부분 텍스트
20181015000183투자위험요소부분 추출
20130122000182html
20130122000182script
20130122000182타켓 스크립트 태그
20130122000182타겟 스크립트 태그의 텍스트
20130122000182본문 부분 텍스트
20130122000182투자위험요소부분 추출
20140715000219html
20140715000219script
20140715000219타켓 스크립트 태그
20140715000219타겟 스크립트 태그의 텍스트
20140715000219본문 부분 텍스트
20140715000219투자위험요소부분 추출
20180515000471html
20180515000471script
20180515000471타켓 스크립트 태그
20180515000471타겟 스크립트 태그의 텍스트
20180515000471본문 부분 텍스트
20180515000

20130322000684html
20130322000684script
20130322000684타켓 스크립트 태그
20130322000684타겟 스크립트 태그의 텍스트
20130322000684본문 부분 텍스트
20130322000684투자위험요소부분 추출
20161004000143html
20161004000143script
20161004000143타켓 스크립트 태그
20161004000143타겟 스크립트 태그의 텍스트
20161004000143본문 부분 텍스트
20161004000143투자위험요소부분 추출
20150914000008html
20150914000008script
20150914000008타켓 스크립트 태그
20150914000008타겟 스크립트 태그의 텍스트
20150914000008본문 부분 텍스트
20150914000008투자위험요소부분 추출
20161010000595html
20161010000595script
20161010000595타켓 스크립트 태그
20161010000595타겟 스크립트 태그의 텍스트
20161010000595본문 부분 텍스트
20161010000595투자위험요소부분 추출
20181203000108html
20181203000108script
20181203000108타켓 스크립트 태그
20181203000108타겟 스크립트 태그의 텍스트
20181203000108본문 부분 텍스트
20181203000108투자위험요소부분 추출
20141204000267html
20141204000267script
20141204000267타켓 스크립트 태그
20141204000267타겟 스크립트 태그의 텍스트
20141204000267본문 부분 텍스트
20141204000267투자위험요소부분 추출
20100609000043html
20100609000043script
20100609000043타켓 스크립트 태그
20100609000043타겟 스크립트 태그의 텍스트
20100609000043본문 부분 텍스트
20100609000

20131213000097html
20131213000097script
20131213000097타켓 스크립트 태그
20131213000097타겟 스크립트 태그의 텍스트
20131213000097본문 부분 텍스트
20131213000097투자위험요소부분 추출
20160218000068html
20160218000068script
20160218000068타켓 스크립트 태그
20160218000068타겟 스크립트 태그의 텍스트
20160218000068본문 부분 텍스트
20160218000068투자위험요소부분 추출
20171011000282html
20171011000282script
20171011000282타켓 스크립트 태그
20171011000282타겟 스크립트 태그의 텍스트
20171011000282본문 부분 텍스트
20171011000282투자위험요소부분 추출
20100524000072html
20100524000072script
20100524000072타켓 스크립트 태그
20100524000072타겟 스크립트 태그의 텍스트
20100524000072본문 부분 텍스트
20100524000072투자위험요소부분 추출
20171124000037html
20171124000037script
20171124000037타켓 스크립트 태그
20171124000037타겟 스크립트 태그의 텍스트
20171124000037본문 부분 텍스트
20171124000037투자위험요소부분 추출
20150720000144html
20150720000144script
20150720000144타켓 스크립트 태그
20150720000144타겟 스크립트 태그의 텍스트
20150720000144본문 부분 텍스트
20150720000144투자위험요소부분 추출
20150717000081html
20150717000081script
20150717000081타켓 스크립트 태그
20150717000081타겟 스크립트 태그의 텍스트
20150717000081본문 부분 텍스트
20150717000

20110111000061html
20110111000061script
20110111000061타켓 스크립트 태그
20110111000061타겟 스크립트 태그의 텍스트
20110111000061본문 부분 텍스트
20110111000061투자위험요소부분 추출
20101206000032html
20101206000032script
20101206000032타켓 스크립트 태그
20101206000032타겟 스크립트 태그의 텍스트
20101206000032본문 부분 텍스트
20101206000032투자위험요소부분 추출
20140926000126html
20140926000126script
20140926000126타켓 스크립트 태그
20140926000126타겟 스크립트 태그의 텍스트
20140926000126본문 부분 텍스트
20140926000126투자위험요소부분 추출
20181108000221html
20181108000221script
20181108000221타켓 스크립트 태그
20181108000221타겟 스크립트 태그의 텍스트
20181108000221본문 부분 텍스트
20181108000221투자위험요소부분 추출
20101122000005html
20101122000005script
20101122000005타켓 스크립트 태그
20101122000005타겟 스크립트 태그의 텍스트
20101122000005본문 부분 텍스트
20101122000005투자위험요소부분 추출
20181015000477html
20181015000477script
20181015000477타켓 스크립트 태그
20181015000477타겟 스크립트 태그의 텍스트
20181015000477본문 부분 텍스트
20181015000477투자위험요소부분 추출
20150604000289html
20150604000289script
20150604000289타켓 스크립트 태그
20150604000289타겟 스크립트 태그의 텍스트
20150604000289본문 부분 텍스트
20150604000

20140731000103html
20140731000103script
20140731000103타켓 스크립트 태그
20140731000103타겟 스크립트 태그의 텍스트
20140731000103본문 부분 텍스트
20140731000103투자위험요소부분 추출
20151105000186html
20151105000186script
20151105000186타켓 스크립트 태그
20151105000186타겟 스크립트 태그의 텍스트
20151105000186본문 부분 텍스트
20151105000186투자위험요소부분 추출
20181115000032html
20181115000032script
20181115000032타켓 스크립트 태그
20181115000032타겟 스크립트 태그의 텍스트
20181115000032본문 부분 텍스트
20181115000032투자위험요소부분 추출
20111121000114html
20111121000114script
20111121000114타켓 스크립트 태그
20111121000114타겟 스크립트 태그의 텍스트
20111121000114본문 부분 텍스트
20111121000114투자위험요소부분 추출
20110530000024html
20110530000024script
20110530000024타켓 스크립트 태그
20110530000024타겟 스크립트 태그의 텍스트
20110530000024본문 부분 텍스트
20110530000024투자위험요소부분 추출
20181023000171html
20181023000171script
20181023000171타켓 스크립트 태그
20181023000171타겟 스크립트 태그의 텍스트
20181023000171본문 부분 텍스트
20181023000171투자위험요소부분 추출
20140922000324html
20140922000324script
20140922000324타켓 스크립트 태그
20140922000324타겟 스크립트 태그의 텍스트
20140922000324본문 부분 텍스트
20140922000

20120626000248html
20120626000248script
20120626000248타켓 스크립트 태그
20120626000248타겟 스크립트 태그의 텍스트
20120626000248본문 부분 텍스트
20120626000248투자위험요소부분 추출
20141205000268html
20141205000268script
20141205000268타켓 스크립트 태그
20141205000268타겟 스크립트 태그의 텍스트
20141205000268본문 부분 텍스트
20141205000268투자위험요소부분 추출
20100830000339html
20100830000339script
20100830000339타켓 스크립트 태그
20100830000339타겟 스크립트 태그의 텍스트
20100830000339본문 부분 텍스트
20100830000339투자위험요소부분 추출
20171212000092html
20171212000092script
20171212000092타켓 스크립트 태그
20171212000092타겟 스크립트 태그의 텍스트
20171212000092본문 부분 텍스트
20171212000092투자위험요소부분 추출
20170526000522html
20170526000522script
20170526000522타켓 스크립트 태그
20170526000522타겟 스크립트 태그의 텍스트
20170526000522본문 부분 텍스트
20170526000522투자위험요소부분 추출
parameter crawling 끝!


In [83]:
df

,rcp_no,dcm,eleld,offset,length,dtd
0,20161028000334,5332050,16,604285,192393,dart3xsd
1,20100709000281,2723113,16,302030,46836,dart3xsd
2,20100618000008,2706880,16,384629,96322,dart3xsd
3,20160125000066,4924831,16,390995,451397,dart3xsd
4,20110531000084,3068515,16,229519,139695,dart3xsd
5,20170118000323,5418967,16,392699,298920,dart3xsd
6,20130722000030,3858856,16,194869,64340,dart3xsd
7,20181122000017,6391857,16,231033,96953,dart3xsd
8,20141205000127,4406863,16,294857,199179,dart3xsd
9,20181106000213,6371273,16,356274,196056,dart3xsd


In [84]:
section_parameter_df = df.copy()
section_parameter_df.head()
len(section_parameter_df)

521

# Concat rcp_no.xlsx to sectionparameter.xlsx 

In [85]:
temp_rcp = df_rcp_no.set_index(df_rcp_no['rcp_no']).drop('rcp_no',axis=1)
temp_rcp.head()
len(temp_rcp)

521

In [92]:
temp_section_parameter = section_parameter_df.set_index('rcp_no')
temp_section_parameter['종목코드']=temp_rcp['종목코드'].values
temp_section_parameter['회사명']=temp_rcp['회사명'].values
temp_section_parameter = temp_section_parameter[temp_section_parameter['eleld']!='non'].copy()
len(temp_section_parameter)

480

# final section_parameter excel file 저장

In [93]:
parameter_url = temp_section_parameter.copy()
dir= 'section_parameter.xlsx'
parameter_url.to_excel(dir)